In [1]:
import json
import pandas as pd
import numpy as np
import numexpr as ne
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
from dotenv import load_dotenv
from tqdm import tqdm
import torch
import torch.cuda.amp as amp

from src.model.model import save_model, load_inference_model
from src.dataset.df import df_dataset, check_dataset
from src.utils.common import set_seed

from src.experiment.inference import inference
from src.experiment.initialize import init_dataset, init_model, init_exp
from src.utils.metrics import compute_surface_dice_score_from_volume

In [2]:
class cfg:
    debug = False
    check_dataset = False

    # = data CFG ====================================================

    dataset_path = "/kaggle/working/dataset/cropped_xy_256_128_z_5_5/"
    train_dataset = "Base2dDataset"
    negative_sample_rate = 0.05
    # = experiment CFG =================================================

    project = "SenNet"
    exp_name = os.path.basename(os.getcwd())
    notes = "negative_sample=0.05"

    # = model CFG ======================================================

    model_arch = "Unet"
    backbone = "se_resnext50_32x4d"
    in_chans = 5
    target_size = 5

    # = training CFG ===================================================

    epochs = 40

    train_batch_size = 64
    valid_batch_size = train_batch_size

    loss = "DiceLoss"
    metrics = "Dice"
    lr = 5e-5
    num_workers = 12

    # = augmentation ===================================================

    image_size = 256
    train_aug = [
        A.RandomRotate90(p=0.5),
        A.RandomGamma(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.RandomResizedCrop(image_size, image_size, scale=(0.8, 1)),
        A.ShiftScaleRotate(p=0.5),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        ToTensorV2(transpose_mask=True),
    ]

    valid_aug = [
        ToTensorV2(transpose_mask=True),
    ]

    # =============== inference ========================================

    test_dataset = "BaseInferenceDataset"
    stride = image_size // 2
    drop_egde_pixel = 32


load_dotenv("/kaggle/key.env")
set_seed()

In [4]:
def filter_dataset(df):
    # trainのうちlabelが全くないものは90%の確率で除外
    df["random"] = np.random.rand(len(df))
    df = df[(df["sum"] > 0) | (df["fold0"] == "valid") | (df["random"] < cfg.negative_sample_rate)]
    df = df.reset_index(drop=True)
    df = df.drop(["random"], axis=1)
    return df


df = pd.read_csv(f"{cfg.dataset_path}/dataset.csv")
df = filter_dataset(df)
# df = df.sample(1000).reset_index(drop=True)
if cfg.debug:
    df = df.sample(10000).reset_index(drop=True)
display(df)

if cfg.check_dataset:
    check_dataset(df, cfg)

/tmp/ipykernel_27781/4046888049.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{cfg.dataset_path}/dataset.csv")


,image_path,label_path,fname,kidney,x,y,z,std,sum,fold0,fold1
0,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x0_y0_z0_std0039_sum0,kidney_1_dense,0,0,0,39,0,valid,train
1,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x0_y0_z100_std0036_sum0,kidney_1_dense,0,0,100,36,0,valid,train
2,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x0_y0_z105_std0035_sum0,kidney_1_dense,0,0,105,35,0,valid,train
3,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x0_y0_z10_std0039_sum0,kidney_1_dense,0,0,10,39,0,valid,train
4,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x0_y0_z110_std0035_sum0,kidney_1_dense,0,0,110,35,0,valid,train
...,...,...,...,...,...,...,...,...,...,...,...
176652,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x896_y256_z975_std0257_sum899,kidney_3_sparse,896,256,975,257,899,train,NaN
176653,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x896_y256_z980_std0259_sum3087,kidney_3_sparse,896,256,980,259,3087,train,NaN
176654,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x896_y256_z985_std0259_sum5991,kidney_3_sparse,896,256,985,259,5991,train,NaN
176655,/kaggle/working/dataset/cropped_xy_256_128_z_5...,/kaggle/working/dataset/cropped_xy_256_128_z_5...,x896_y256_z990_std0262_sum7311,kidney_3_sparse,896,256,990,262,7311,train,NaN


In [5]:
if cfg.debug:
    print("!!!Debug mode!!!\n")
    cfg.epochs = 5

for fold in range(2):
    train_dataloader, valid_dataloader = init_dataset(fold, df, cfg)
    model, scaler, criterion, optimizer, scheduler, metrics = init_model(cfg)
    slacknotify = init_exp(fold, cfg)

    path_best = f"./{cfg.exp_name}/{cfg.exp_name}_best_fold{fold}.pth"
    path_last = f"./{cfg.exp_name}/{cfg.exp_name}_last_fold{fold}.pth"

    best_loss = float("inf")
    for epoch in range(cfg.epochs):
        model.train()
        total_loss = 0.0
        pbar_train = tqdm(enumerate(train_dataloader), total=len(train_dataloader), bar_format="{l_bar}{bar:10}{r_bar}{bar:-0b}")

        for i, (images, masks) in pbar_train:
            images, masks = images.cuda(), masks.cuda()
            optimizer.zero_grad()

            with amp.autocast():
                preds = model(images)
                loss = criterion(preds, masks)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.detach().item()

            loss_ = total_loss / (i + 1)
            lr = f"LR : {scheduler.get_lr()[0]:.2E}"
            gpu_mem = f"Mem : {torch.cuda.memory_reserved() / 1E9:.3g}GB"
            pbar_train.set_description(("%10s  " * 3 + "%10s") % (f"Epoch {epoch}/{cfg.epochs}", gpu_mem, lr, f"Loss: {loss_:.4f}"))

        train_loss = loss_
        scheduler.step()
        wandb.log({"epoch": epoch, "train_loss": train_loss})

        model.eval()
        total_loss = 0.0
        pbar_val = tqdm(enumerate(valid_dataloader), total=len(valid_dataloader), bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}")

        for i, (images, masks) in pbar_val:
            images, masks = images.cuda(), masks.cuda()
            with torch.no_grad():
                preds = model(images)
                loss = criterion(preds, masks)
                total_loss += loss.item()

            loss_ = total_loss / (i + 1)
            pbar_val.set_description(("%10s") % (f"Val Loss: {loss_:.4f}"))
        valid_loss = loss_
        wandb.log({"epoch": epoch, "valid_loss": valid_loss})

        if valid_loss < best_loss:
            print(f"loss : {valid_loss:.4f}\tSAVED MODEL\n")
            slacknotify.send_reply(f"epoch : {epoch}\tscore : {valid_loss:.4f}\tBEST")
            best_loss = valid_loss
            save_model(model, cfg, path_best, loss=loss)
        else:
            print(f"loss : {valid_loss:.4f}\n")
            slacknotify.send_reply(f"epoch : {epoch}\tscore : {valid_loss:.4f}")

    save_model(model, cfg, path_last, loss=valid_loss)
    wandb.config.update({"last_loss": valid_loss, "best_loss": best_loss})

    slacknotify.send_reply(f"{cfg.exp_name}_fold{fold} training finished\nbest loss : {best_loss:.4f} last loss : {loss_:.4f}", True)

    if wandb.run:
        wandb.finish()

model_arch:  Unet
backbone:  se_resnext50_32x4d


wandb: Currently logged in as: welshonionman. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/wandb/run-20240106_152156-134as7zy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run exp001_fold0
wandb: ⭐️ View project at https://wandb.ai/welshonionman/SenNet
wandb: 🚀 View run at https://wandb.ai/welshonionman/SenNet/runs/134as7zy
Epoch 0/40  Mem : 10.1GB  LR : 5.00E-05  Loss: 0.6450: 100%|██████████| 1291/1291 [04:19<00:00,  4.98it/s]
Val Loss: 0.5472: 100%|██████████| 1471/1471 [03:04<00:00,  7.98it/s]                                                                                                                                                                                                                                  


loss : 0.5472	SAVED MODEL



Epoch 1/40  Mem : 11.9GB  LR : 5.00E-04  Loss: 0.1271: 100%|██████████| 1291/1291 [04:16<00:00,  5.04it/s]
Val Loss: 0.3990: 100%|██████████| 1471/1471 [03:03<00:00,  8.00it/s]                                                                                                                                                                                                                                  


loss : 0.3990	SAVED MODEL



...

Epoch 38/40  Mem : 11.9GB  LR : 1.32E-06  Loss: 0.0744: 100%|██████████| 2329/2329 [07:53<00:00,  4.91it/s]
Val Loss: 0.1298: 100%|██████████| 215/215 [00:27<00:00,  7.91it/s]                                                                                                                                                                                                                                    


loss : 0.1298



Epoch 39/40  Mem : 11.9GB  LR : 7.89E-07  Loss: 0.0746: 100%|██████████| 2329/2329 [07:53<00:00,  4.92it/s]
Val Loss: 0.1303: 100%|██████████| 215/215 [00:26<00:00,  7.99it/s]                                                                                                                                                                                                                                    


loss : 0.1303



wandb: WARNING No program path found, not creating job artifact. See https://docs.wandb.ai/guides/launch/create-job
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb: train_loss █▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: valid_loss █▂▂▂▂▂▂▁▂▂▁▁▁▁▂▁▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:      epoch 39
wandb: train_loss 0.07458
wandb: valid_loss 0.13031
wandb: 
wandb: 🚀 View run exp001_fold1 at: https://wandb.ai/welshonionman/SenNet/runs/ov77yn9d
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: /kaggle/wandb/run-20240106_201654-ov77yn9d/logs


In [5]:
fold_dict = json.load(open("/kaggle/src/dataset/fold.json", "r"))

for fold in range(2):
    print(f"fold_{fold}")
    valid_kidney = fold_dict[f"fold{fold}"]["valid"][0]
    path_best = f"./{cfg.exp_name}/{cfg.exp_name}_best_fold{fold}.pth"
    # model = load_inference_model(path_best, cfg)

    stack_path = f"/kaggle/working/dataset/stack_train01/{valid_kidney}_images.npy"
    label_path = f"/kaggle/working/dataset/stack_train01/{valid_kidney}_labels.npy"
    save_path = "./preds"
    preds_path = f"{save_path}/{valid_kidney}_preds.npy"
    # inference(model, stack_path, save_path, cfg)

    label = np.load(label_path)
    preds = np.load(preds_path)
    thresh_score_dict = {}

    for thresh in np.arange(0.1, 0.99, 0.1):
        thresh = round(thresh, 5)
        thresh_score_dict[thresh] = compute_surface_dice_score_from_volume(ne.evaluate("preds > thresh"), label)

    max_thresh = max(thresh_score_dict, key=thresh_score_dict.get)

    for thresh in np.arange(max_thresh - 0.1, max_thresh + 0.1, 0.01):
        thresh = round(thresh, 5)
        thresh_score_dict[thresh] = compute_surface_dice_score_from_volume(ne.evaluate("preds > thresh"), label)

    print(max(thresh_score_dict.items(), key=lambda x: x[1]))

fold_0
(0.5, 0.8012)
fold_1
(0.33, 0.8824)


/kaggle/working/notebook/experiment/exp001
